<h1 style="color:#D30982;">Advanced Features of qBraid Lab</h1>

Now we come to introducing some of the more recent features added to qBraid Lab to elevate the development/research experience. Some of these features are free (qBraid-sdk, code-snippets), some require certain levels of subscription (share environments/notebooks, omnisearch), and others are in closed beta (GPU-integration). For all participants in the IEEE QCE23 tutorial, we are providing limited access to all of these features at no cost. Simply enter the code 'XXXXXX' at https://account.qbraid.com/account-details and then open a lab instance.

In this notebook, you will explore: <br>
1. <a href="#qbraid-sdk">Instructions on how to use the recently open-sourced qBraid-sdk to do cross-framework abstraction, transpilation, and execution of quantum programs. If interested, you are encouraged to participate in the open-source community development of the qBraid-sdk here -> https://github.com/qBraid/qBraid</a> <br>
2. <a href="#code-snippets">Instructions on how to utilize code snippets.</a> <br>
3. <a href="#sharing-environments-and-notebooks">Instructions on how to share environments and notebooks for more seamless collaboration.</a> <br>
4. <a href="#GPU-integration">Instructions on how to utilize GPU-acceleration in quantum workflows.</a> <br>

<h2> qBraid-SDK </h2> <a name="qbraid-sdk"></a>


In [1]:
# Construct a quantum program of any supported program type:

from qbraid import QPROGRAM_LIBS
from qbraid.interface import random_circuit

QPROGRAM_LIBS

['braket', 'cirq', 'qiskit', 'pyquil', 'pytket', 'qasm2', 'qasm3']

In [2]:
circuit = random_circuit("qiskit", num_qubits=1, measure=True)

In [3]:
from qbraid import get_devices

from qbraid.api import ibm_least_busy_qpu

get_devices(filters={"architecture": {"$regex": "superconducting"}, "vendor": "AWS"})

ImportError: cannot import name 'ibm_least_busy_qpu' from 'qbraid.api' (/home/kenny/anaconda3/lib/python3.11/site-packages/qbraid/api/__init__.py)

In [4]:
from qbraid import device_wrapper

jobs  = []

qbraid_ids = ['aws_oqc_lucy', 'ibm_q_perth']

for device in qbraid_ids:
    qdevice = device_wrapper(device)
    qjob = qdevice.run(circuit, shots=1000)
    jobs.append(qjob)

KeyError: 'aws.device'

In [ ]:
from qbraid import get_jobs

get_jobs(filters={"numResults": 2})

In [ ]:
print("{:<20} {:<20}".format('Device','Counts'))

for i, job in enumerate(jobs):
    result = job.result()
    counts = result.measurement_counts()
    print("{:<20} {:<20}".format(qbraid_ids[i],str(counts)))

Here, you will learn how to use the qBraid SDK to interface with quantum circuit objects accross various frontends. We will demonstrate how to use the transpiler to convert circuits between packages, and highlight a few other circuit-based convenience features.

In [6]:
from qbraid import QPROGRAM_TYPES

for k in QPROGRAM_TYPES:
    print(k)

<class 'braket.circuits.circuit.Circuit'>
<class 'cirq.circuits.circuit.Circuit'>
<class 'qiskit.circuit.quantumcircuit.QuantumCircuit'>
<class 'pyquil.quil.Program'>
<class 'pytket._tket.circuit.Circuit'>
<class 'str'>


In [7]:
from qiskit import QuantumCircuit

def bell():
    circuit = QuantumCircuit(2)
    circuit.h(0)
    circuit.cx(0,1)
    return circuit

In [8]:
qiskit_circuit = bell()
qiskit_circuit.draw()

┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘

In [9]:
from qbraid import circuit_wrapper

qprogram = circuit_wrapper(qiskit_circuit)

QbraidError: Error applying circuit wrapper to quantum program of type <class 'qiskit.circuit.quantumcircuit.QuantumCircuit'>

In [10]:
from qbraid import QPROGRAM_LIBS
QPROGRAM_LIBS

['braket', 'cirq', 'qiskit', 'pyquil', 'pytket', 'qasm2', 'qasm3']

In [12]:
from qbraid import circuit_wrapper
from qbraid.interface import random_circuit
qiskit_circuit = random_circuit("cirq")
cirq_circuit = circuit_wrapper(qiskit_circuit).transpile("qiskit")
print(qiskit_circuit)


QbraidError: Error applying circuit wrapper to quantum program of type <class 'cirq.circuits.circuit.Circuit'>